In [1]:
%reload_ext autoreload
%autoreload 2
#show all columnsdf
#ignore warnings.

In [8]:
import pandas as pd
pd.set_option('display.max_columns', 9000)
import numpy as np
import os, sys
import importlib
import yaml, json
from pathlib import Path
base=Path('./Data/NSF-Downloads-8july2024/')

In [ ]:
import pandas as pd
import zipfile
from pathlib import Path

def process_zip_files(base_path, output_path, min_years):
    """
    Processes ZIP files in the given base directory, extracts XML files, and returns a DataFrame
    with information about the extracted files for years > min_years.
    
    Parameters:
        base_path (Path): The base directory containing ZIP files.
        output_path (Path): The directory where files will be temporarily extracted.
        min_years (int): Minimum year to include in the results.
    
    Returns:
        pd.DataFrame: A DataFrame with columns ['file', 'year', 'xmlfiles'].
    """
    base = Path(base_path)
    output = Path(output_path)
    output.mkdir(parents=True, exist_ok=True)  # Ensure output directory exists
    
    zipfiles = list(base.rglob("*.zip"))  # List all ZIP files in base directory
    details = []
    
    for file in zipfiles:
        file_info = {}
        file_info['file'] = str(file)
        
        # Extract year from filename (assuming the year is in the filename)
        try:
            year = int(file.stem.split(".")[0])  # Adjust split logic based on actual filename format
            file_info['year'] = int(year)
        except ValueError:
            print(f"Could not extract year from filename: {file}")
            continue
        
        # Skip files with years <= min_years
        if year <= min_years:
            continue
        
        # Create a year-specific directory for extraction
        year_dir = output / str(year)
        year_dir.mkdir(parents=True, exist_ok=True)
        
        # Extract ZIP file contents to the year-specific directory
        try:
            with zipfile.ZipFile(file, 'r') as zip_ref:
                zip_ref.extractall(year_dir)
            file_info['errors']=True
        except Exception as e:
            print(f"Failed to extract {file}: {e}")
            file_info['errors']=False
            continue
        
        # Count the number of XML files in the year-specific directory
        xmlfiles = list(year_dir.rglob("*.xml"))
        file_info['xmlfiles'] = len(xmlfiles)
        
        # Append file info to the details list
        details.append(file_info)
    
    # Convert the details list into a DataFrame
    df = pd.DataFrame(details)
    #sort by year decending
    df.sort_values(by='year', ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# Example usage
base_path = './Data/NSF-Downloads-8july2024/'
output_path = './temp'
min_years = 1999  # Replace with your minimum year threshold

df = process_zip_files(base_path, output_path, min_years)
df


,file,year,errors,xmlfiles
0,Data/NSF-Downloads-8july2024/2024.zip,2024,True,8199
1,Data/NSF-Downloads-8july2024/2018.zip,2018,True,12235
2,Data/NSF-Downloads-8july2024/2019.zip,2019,True,13043
3,Data/NSF-Downloads-8july2024/2009.zip,2009,True,15430
4,Data/NSF-Downloads-8july2024/2021.zip,2021,True,12652
5,Data/NSF-Downloads-8july2024/2020.zip,2020,True,13300
6,Data/NSF-Downloads-8july2024/2008.zip,2008,True,12612
7,Data/NSF-Downloads-8july2024/2022.zip,2022,True,11472
8,Data/NSF-Downloads-8july2024/2023.zip,2023,True,11130
9,Data/NSF-Downloads-8july2024/2012.zip,2012,True,12165


In [3]:
df.sort_values(by='year', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,file,year,errors,xmlfiles
0,Data/NSF-Downloads-8july2024/2024.zip,2024,True,8199
1,Data/NSF-Downloads-8july2024/2023.zip,2023,True,11130
2,Data/NSF-Downloads-8july2024/2022.zip,2022,True,11472
3,Data/NSF-Downloads-8july2024/2021.zip,2021,True,12652
4,Data/NSF-Downloads-8july2024/2020.zip,2020,True,13300
5,Data/NSF-Downloads-8july2024/2019.zip,2019,True,13043
6,Data/NSF-Downloads-8july2024/2018.zip,2018,True,12235
7,Data/NSF-Downloads-8july2024/2017.zip,2017,True,12289
8,Data/NSF-Downloads-8july2024/2016.zip,2016,True,12594
9,Data/NSF-Downloads-8july2024/2015.zip,2015,True,13072


In [4]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
from datetime import datetime
import pandas as pd
def parse_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()

        data = {
            'AwardID': root.findtext('.//AwardID'),
            'AwardTitle': root.findtext('.//AwardTitle'),
            'Agency': root.findtext('.//AGENCY'),
            'AwardEffectiveDate': root.findtext('.//AwardEffectiveDate'),
            'AwardExpirationDate': root.findtext('.//AwardExpirationDate'),
            'AwardTotalIntnAmount': root.findtext('.//AwardTotalIntnAmount'),
            'AwardAmount': root.findtext('.//AwardAmount'),
            'AwardInstrument': root.findtext('.//AwardInstrument/Value'),
            'Organization_Code': root.findtext('.//Organization/Code'),
            'Directorate_Abbreviation': root.findtext('.//Directorate/Abbreviation'),
            'Directorate_LongName': root.findtext('.//Directorate/LongName'),
            'Division_Abbreviation': root.findtext('.//Division/Abbreviation'),
            'Division_LongName': root.findtext('.//Division/LongName'),
            'ProgramOfficer_Name': root.findtext('.//ProgramOfficer/SignBlockName'),
            'ProgramOfficer_Email': root.findtext('.//ProgramOfficer/PO_EMAI'),
            'ProgramOfficer_Phone': root.findtext('.//ProgramOfficer/PO_PHON'),
            'AbstractNarration': root.findtext('.//AbstractNarration'),
            'MinAmdLetterDate': root.findtext('.//MinAmdLetterDate'),
            'MaxAmdLetterDate': root.findtext('.//MaxAmdLetterDate'),
            'ARRAAmount': root.findtext('.//ARRAAmount'),
            'TRAN_TYPE': root.findtext('.//TRAN_TYPE'),
            'CFDA_NUM': root.findtext('.//CFDA_NUM'),
            'NSF_PAR_USE_FLAG': root.findtext('.//NSF_PAR_USE_FLAG'),
            'FUND_AGCY_CODE': root.findtext('.//FUND_AGCY_CODE'),
            'AWDG_AGCY_CODE': root.findtext('.//AWDG_AGCY_CODE'),
            'Investigator1_FirstName': root.findtext('.//Investigator[1]/FirstName'),
            'Investigator1_LastName': root.findtext('.//Investigator[1]/LastName'),
            'Investigator1_MiddleInitial': root.findtext('.//Investigator[1]/PI_MID_INIT'),
            'Investigator1_Suffix': root.findtext('.//Investigator[1]/PI_SUFX_NAME'),
            'Investigator1_FullName': root.findtext('.//Investigator[1]/PI_FULL_NAME'),
            'Investigator1_Email': root.findtext('.//Investigator[1]/EmailAddress'),
            'Investigator1_NSFID': root.findtext('.//Investigator[1]/NSF_ID'),
            'Investigator1_StartDate': root.findtext('.//Investigator[1]/StartDate'),
            'Investigator1_EndDate': root.findtext('.//Investigator[1]/EndDate'),
            'Investigator1_RoleCode': root.findtext('.//Investigator[1]/RoleCode'),
            'Investigator2_FirstName': root.findtext('.//Investigator[2]/FirstName'),
            'Investigator2_LastName': root.findtext('.//Investigator[2]/LastName'),
            'Investigator2_MiddleInitial': root.findtext('.//Investigator[2]/PI_MID_INIT'),
            'Investigator2_Suffix': root.findtext('.//Investigator[2]/PI_SUFX_NAME'),
            'Investigator2_FullName': root.findtext('.//Investigator[2]/PI_FULL_NAME'),
            'Investigator2_Email': root.findtext('.//Investigator[2]/EmailAddress'),
            'Investigator2_NSFID': root.findtext('.//Investigator[2]/NSF_ID'),
            'Investigator2_StartDate': root.findtext('.//Investigator[2]/StartDate'),
            'Investigator2_EndDate': root.findtext('.//Investigator[2]/EndDate'),
            'Investigator2_RoleCode': root.findtext('.//Investigator[2]/RoleCode'),
            'Investigator3_FirstName': root.findtext('.//Investigator[3]/FirstName'),
            'Investigator3_LastName': root.findtext('.//Investigator[3]/LastName'),
            'Investigator3_MiddleInitial': root.findtext('.//Investigator[3]/PI_MID_INIT'),
            'Investigator3_Suffix': root.findtext('.//Investigator[3]/PI_SUFX_NAME'),
            'Investigator3_FullName': root.findtext('.//Investigator[3]/PI_FULL_NAME'),
            'Investigator3_Email': root.findtext('.//Investigator[3]/EmailAddress'),
            'Investigator3_NSFID': root.findtext('.//Investigator[3]/NSF_ID'),
            'Investigator3_StartDate': root.findtext('.//Investigator[3]/StartDate'),
            'Investigator3_EndDate': root.findtext('.//Investigator[3]/EndDate'),
            'Investigator3_RoleCode': root.findtext('.//Investigator[3]/RoleCode'),
            'Institution_Name': root.findtext('.//Institution/Name'),
            'Institution_City': root.findtext('.//Institution/CityName'),
            'Institution_State': root.findtext('.//Institution/StateName'),
            'Institution_Zip': root.findtext('.//Institution/ZipCode'),
            'Institution_Country': root.findtext('.//Institution/CountryName'),
            'Institution_Phone': root.findtext('.//Institution/PhoneNumber'),
            'Institution_StreetAddress1': root.findtext('.//Institution/StreetAddress'),
            'Institution_StreetAddress2': root.findtext('.//Institution/StreetAddress2'),
            'Institution_CongressDistrict': root.findtext('.//Institution/CONGRESSDISTRICT'),
            'Institution_CongressDistrictOrg': root.findtext('.//Institution/CONGRESS_DISTRICT_ORG'),
            'Institution_OrgUEINum': root.findtext('.//Institution/ORG_UEI_NUM'),
            'Institution_OrgLglBusName': root.findtext('.//Institution/ORG_LGL_BUS_NAME'),
            'Performance_Institution_Name': root.findtext('.//Performance_Institution/Name'),
            'Performance_Institution_City': root.findtext('.//Performance_Institution/CityName'),
            'Performance_Institution_State': root.findtext('.//Performance_Institution/StateName'),
            'Performance_Institution_Zip': root.findtext('.//Performance_Institution/ZipCode'),
            'Performance_Institution_StreetAddress': root.findtext('.//Performance_Institution/StreetAddress'),
            'Performance_Institution_CountryCode': root.findtext('.//Performance_Institution/CountryCode'),
            'Performance_Institution_Country': root.findtext('.//Performance_Institution/CountryName'),
            'Performance_Institution_State': root.findtext('.//Performance_Institution/StateName'),
            'Performance_Institution_CountryFlag': root.findtext('.//Performance_Institution/CountryFlag'),
            'Performance_Institution_CongressDistrict': root.findtext('.//Performance_Institution/CONGRESSDISTRICT'),
            'Performance_Institution_CongressDistrictPerf': root.findtext('.//Performance_Institution/CONGRESS_DISTRICT_PERF'),
            'ProgramElement1_Code': root.findtext('.//ProgramElement[1]/Code'),
            'ProgramElement1_Text': root.findtext('.//ProgramElement[1]/Text'),
            'ProgramElement2_Code': root.findtext('.//ProgramElement[2]/Code'),
            'ProgramElement2_Text': root.findtext('.//ProgramElement[2]/Text'),
            'ProgramElement3_Code': root.findtext('.//ProgramElement[3]/Code'),
            'ProgramElement3_Text': root.findtext('.//ProgramElement[3]/Text'),
            'ProgramReference_Code': root.findtext('.//ProgramReference/Code'),
            'ProgramReference_Text': root.findtext('.//ProgramReference/Text'),
            'Appropriation_Code': root.findtext('.//Appropriation/Code'),
            'Appropriation_Name': root.findtext('.//Appropriation/Name'),
            'Appropriation_SymbolID': root.findtext('.//Appropriation/APP_SYMB_ID'),
            'Fund_Code': root.findtext('.//Fund/Code'),
            'Fund_Name': root.findtext('.//Fund/Name'),
            'Fund_SymbolID': root.findtext('.//Fund/FUND_SYMB_ID'),
            'Fund_Obligation': root.findtext('.//FUND_OBLG')
        }
        return data
    except ET.ParseError:
        print(f'Error parsing file: {file_path}')
        return None


In [5]:
import pandas as pd
from pathlib import Path
from datetime import datetime

def collect_data_from_xml_files(root_folder, year):
    data_list = []
    error_files = []
    yr_folder = root_folder / str(year)
    xmlfiles = list(yr_folder.rglob("*.xml"))
    
    for filename in xmlfiles:
        # Assuming parse_xml is a user-defined function to parse XML files
        data = parse_xml(filename)  # Use filename directly as it contains the full path
        
        if data is not None:
            data['Year'] = year
            data_list.append(data)
        else:
            error_files.append(filename)
    
    return pd.DataFrame(data_list), error_files

# Define the root folder and output directories
root_folder = Path('./temp')
output_csv = Path('./outputcsv')
output_pkl = Path('./outputpkl')

# Ensure output directories exist
output_csv.mkdir(parents=True, exist_ok=True)
output_pkl.mkdir(parents=True, exist_ok=True)

# Add new columns to the DataFrame
df['df_rows'] = 0
df['xml_errors'] = 0
df['xml_error_list'] = None
df['time_parsed'] = None

# Iterate through the DataFrame and process XML files
for index, row in df.iterrows():
    print(f"Processing year: {row['year']} with {row['xmlfiles']} XML files...")
    
    # Collect data and errors from XML files
    data, error_files = collect_data_from_xml_files(root_folder, row['year'])
    
    # Update the DataFrame with new information
    df.loc[index, 'df_rows'] = len(data)
    df.loc[index, 'xml_errors'] = len(error_files)
    df.loc[index, 'xml_error_list'] = str(error_files)
    df.loc[index, 'time_parsed'] = datetime.now()
    
    # Save the processed data
    if not data.empty:
        data.to_csv(output_csv / f'{row["year"]}.csv', index=False)
        data.to_pickle(output_pkl / f'{row["year"]}.pkl')

# Output the final DataFrame
df


Processing year: 2024 with 8199 XML files...
Processing year: 2023 with 11130 XML files...
Processing year: 2022 with 11472 XML files...
Error parsing file: temp/2022/49100422C0020.xml
Processing year: 2021 with 12652 XML files...
Processing year: 2020 with 13300 XML files...
Processing year: 2019 with 13043 XML files...
Processing year: 2018 with 12235 XML files...
Processing year: 2017 with 12289 XML files...
Processing year: 2016 with 12594 XML files...
Processing year: 2015 with 13072 XML files...
Processing year: 2014 with 12109 XML files...
Processing year: 2013 with 11502 XML files...
Processing year: 2012 with 12165 XML files...
Error parsing file: temp/2012/1201835.xml
Processing year: 2011 with 11652 XML files...
Error parsing file: temp/2011/1147930.xml
Error parsing file: temp/2011/1127340.xml
Error parsing file: temp/2011/1117597.xml
Error parsing file: temp/2011/1116081.xml
Processing year: 2010 with 13094 XML files...
Error parsing file: temp/2010/1066456.xml
Error parsi

,file,year,errors,xmlfiles,df_rows,xml_errors,xml_error_list,time_parsed
0,Data/NSF-Downloads-8july2024/2024.zip,2024,True,8199,8199,0,[],2025-01-20 14:40:26.892142
1,Data/NSF-Downloads-8july2024/2023.zip,2023,True,11130,11130,0,[],2025-01-20 14:40:34.954554
2,Data/NSF-Downloads-8july2024/2022.zip,2022,True,11472,11471,1,[PosixPath('temp/2022/49100422C0020.xml')],2025-01-20 14:40:42.969165
3,Data/NSF-Downloads-8july2024/2021.zip,2021,True,12652,12652,0,[],2025-01-20 14:40:51.825242
4,Data/NSF-Downloads-8july2024/2020.zip,2020,True,13300,13300,0,[],2025-01-20 14:41:01.387771
5,Data/NSF-Downloads-8july2024/2019.zip,2019,True,13043,13043,0,[],2025-01-20 14:41:11.547159
6,Data/NSF-Downloads-8july2024/2018.zip,2018,True,12235,12235,0,[],2025-01-20 14:41:20.727974
7,Data/NSF-Downloads-8july2024/2017.zip,2017,True,12289,12289,0,[],2025-01-20 14:41:29.909602
8,Data/NSF-Downloads-8july2024/2016.zip,2016,True,12594,12594,0,[],2025-01-20 14:41:40.477871
9,Data/NSF-Downloads-8july2024/2015.zip,2015,True,13072,13072,0,[],2025-01-20 14:41:50.074970


In [6]:
df.to_csv('0_processing.csv', index=False)

In [ ]:
#iterate over the df and read the pickle files to a single dataframe
all=pd.DataFrame()
for index, row in df.iterrows():
    print(f"Processing year: {row['year']} with {row['xmlfiles']} XML files...")
    tmp=pd.read_pickle(output_pkl / f'{row["year"]}.pkl')
    all=pd.concat([all, tmp])
all.to_csv(output_csv/'all.csv', index=False)  
all.to_pickle(output_pkl/'all.pkl')   

Processing year: 2024 with 8199 XML files...


TypeError: NDFrame.to_pickle() got an unexpected keyword argument 'index'

In [11]:

def collect_data_from_xml_files(root_folder, year):
    data_list = []
    error_files = []
    yr_folder=root_folder/str(year)
    xmlfiles = list(yr_folder.rglob("*.xml"))
    for filename in xmlfiles:
        data = parse_xml(yr_folder/filename)

        if data is not None:
            data['Year'] = year
            data_list.append(data)
        else:
            error_files.append(file_path)

    return pd.DataFrame(data_list), error_files

# Define the root folder where the year folders are located
root_folder = Path('./temp')
output_csv = Path('./outputcsv')
output_csv = Path('./outputpkl')
#iterate through the dataframe df
for index, row in df.iterrows():
    print(row['year'], row['xmlfiles'])
# Collect data and create DataFrame
    data, error_files = collect_data_from_xml_files(root_folder, row['year'])
    #create the directory if it doesn't exist ./output/csv
    output_csv.mkdir(parents=True, exist_ok=True)
    output_pkl.mkdir(parents=True, exist_ok=True)
    df.loc[index, 'df_rows'] = len(data)
    df.loc[index, 'xml_errors'] = len(error_files)
    df.loc[index, 'xml_error_list'] = len(errors)
    df['time_parsed'] = datetime.now()
    data.to_csv(output_csv/f'{row["year"]}.csv', index=False)
    data.to_pickle(output_pkl/f'{row["year"]}.pkl')
    
    # Get the current date and time
    
df
    

2024 8199


FileNotFoundError: [Errno 2] No such file or directory: 'temp/2024/temp/2024/2428698.xml'

In [5]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
from datetime import datetime
import pandas as pd
def parse_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()

        data = {
            'AwardID': root.findtext('.//AwardID'),
            'AwardTitle': root.findtext('.//AwardTitle'),
            'Agency': root.findtext('.//AGENCY'),
            'AwardEffectiveDate': root.findtext('.//AwardEffectiveDate'),
            'AwardExpirationDate': root.findtext('.//AwardExpirationDate'),
            'AwardTotalIntnAmount': root.findtext('.//AwardTotalIntnAmount'),
            'AwardAmount': root.findtext('.//AwardAmount'),
            'AwardInstrument': root.findtext('.//AwardInstrument/Value'),
            'Organization_Code': root.findtext('.//Organization/Code'),
            'Directorate_Abbreviation': root.findtext('.//Directorate/Abbreviation'),
            'Directorate_LongName': root.findtext('.//Directorate/LongName'),
            'Division_Abbreviation': root.findtext('.//Division/Abbreviation'),
            'Division_LongName': root.findtext('.//Division/LongName'),
            'ProgramOfficer_Name': root.findtext('.//ProgramOfficer/SignBlockName'),
            'ProgramOfficer_Email': root.findtext('.//ProgramOfficer/PO_EMAI'),
            'ProgramOfficer_Phone': root.findtext('.//ProgramOfficer/PO_PHON'),
            'AbstractNarration': root.findtext('.//AbstractNarration'),
            'MinAmdLetterDate': root.findtext('.//MinAmdLetterDate'),
            'MaxAmdLetterDate': root.findtext('.//MaxAmdLetterDate'),
            'ARRAAmount': root.findtext('.//ARRAAmount'),
            'TRAN_TYPE': root.findtext('.//TRAN_TYPE'),
            'CFDA_NUM': root.findtext('.//CFDA_NUM'),
            'NSF_PAR_USE_FLAG': root.findtext('.//NSF_PAR_USE_FLAG'),
            'FUND_AGCY_CODE': root.findtext('.//FUND_AGCY_CODE'),
            'AWDG_AGCY_CODE': root.findtext('.//AWDG_AGCY_CODE'),
            'Investigator1_FirstName': root.findtext('.//Investigator[1]/FirstName'),
            'Investigator1_LastName': root.findtext('.//Investigator[1]/LastName'),
            'Investigator1_MiddleInitial': root.findtext('.//Investigator[1]/PI_MID_INIT'),
            'Investigator1_Suffix': root.findtext('.//Investigator[1]/PI_SUFX_NAME'),
            'Investigator1_FullName': root.findtext('.//Investigator[1]/PI_FULL_NAME'),
            'Investigator1_Email': root.findtext('.//Investigator[1]/EmailAddress'),
            'Investigator1_NSFID': root.findtext('.//Investigator[1]/NSF_ID'),
            'Investigator1_StartDate': root.findtext('.//Investigator[1]/StartDate'),
            'Investigator1_EndDate': root.findtext('.//Investigator[1]/EndDate'),
            'Investigator1_RoleCode': root.findtext('.//Investigator[1]/RoleCode'),
            'Investigator2_FirstName': root.findtext('.//Investigator[2]/FirstName'),
            'Investigator2_LastName': root.findtext('.//Investigator[2]/LastName'),
            'Investigator2_MiddleInitial': root.findtext('.//Investigator[2]/PI_MID_INIT'),
            'Investigator2_Suffix': root.findtext('.//Investigator[2]/PI_SUFX_NAME'),
            'Investigator2_FullName': root.findtext('.//Investigator[2]/PI_FULL_NAME'),
            'Investigator2_Email': root.findtext('.//Investigator[2]/EmailAddress'),
            'Investigator2_NSFID': root.findtext('.//Investigator[2]/NSF_ID'),
            'Investigator2_StartDate': root.findtext('.//Investigator[2]/StartDate'),
            'Investigator2_EndDate': root.findtext('.//Investigator[2]/EndDate'),
            'Investigator2_RoleCode': root.findtext('.//Investigator[2]/RoleCode'),
            'Investigator3_FirstName': root.findtext('.//Investigator[3]/FirstName'),
            'Investigator3_LastName': root.findtext('.//Investigator[3]/LastName'),
            'Investigator3_MiddleInitial': root.findtext('.//Investigator[3]/PI_MID_INIT'),
            'Investigator3_Suffix': root.findtext('.//Investigator[3]/PI_SUFX_NAME'),
            'Investigator3_FullName': root.findtext('.//Investigator[3]/PI_FULL_NAME'),
            'Investigator3_Email': root.findtext('.//Investigator[3]/EmailAddress'),
            'Investigator3_NSFID': root.findtext('.//Investigator[3]/NSF_ID'),
            'Investigator3_StartDate': root.findtext('.//Investigator[3]/StartDate'),
            'Investigator3_EndDate': root.findtext('.//Investigator[3]/EndDate'),
            'Investigator3_RoleCode': root.findtext('.//Investigator[3]/RoleCode'),
            'Institution_Name': root.findtext('.//Institution/Name'),
            'Institution_City': root.findtext('.//Institution/CityName'),
            'Institution_State': root.findtext('.//Institution/StateName'),
            'Institution_Zip': root.findtext('.//Institution/ZipCode'),
            'Institution_Country': root.findtext('.//Institution/CountryName'),
            'Institution_Phone': root.findtext('.//Institution/PhoneNumber'),
            'Institution_StreetAddress1': root.findtext('.//Institution/StreetAddress'),
            'Institution_StreetAddress2': root.findtext('.//Institution/StreetAddress2'),
            'Institution_CongressDistrict': root.findtext('.//Institution/CONGRESSDISTRICT'),
            'Institution_CongressDistrictOrg': root.findtext('.//Institution/CONGRESS_DISTRICT_ORG'),
            'Institution_OrgUEINum': root.findtext('.//Institution/ORG_UEI_NUM'),
            'Institution_OrgLglBusName': root.findtext('.//Institution/ORG_LGL_BUS_NAME'),
            'Performance_Institution_Name': root.findtext('.//Performance_Institution/Name'),
            'Performance_Institution_City': root.findtext('.//Performance_Institution/CityName'),
            'Performance_Institution_State': root.findtext('.//Performance_Institution/StateName'),
            'Performance_Institution_Zip': root.findtext('.//Performance_Institution/ZipCode'),
            'Performance_Institution_StreetAddress': root.findtext('.//Performance_Institution/StreetAddress'),
            'Performance_Institution_CountryCode': root.findtext('.//Performance_Institution/CountryCode'),
            'Performance_Institution_Country': root.findtext('.//Performance_Institution/CountryName'),
            'Performance_Institution_State': root.findtext('.//Performance_Institution/StateName'),
            'Performance_Institution_CountryFlag': root.findtext('.//Performance_Institution/CountryFlag'),
            'Performance_Institution_CongressDistrict': root.findtext('.//Performance_Institution/CONGRESSDISTRICT'),
            'Performance_Institution_CongressDistrictPerf': root.findtext('.//Performance_Institution/CONGRESS_DISTRICT_PERF'),
            'ProgramElement1_Code': root.findtext('.//ProgramElement[1]/Code'),
            'ProgramElement1_Text': root.findtext('.//ProgramElement[1]/Text'),
            'ProgramElement2_Code': root.findtext('.//ProgramElement[2]/Code'),
            'ProgramElement2_Text': root.findtext('.//ProgramElement[2]/Text'),
            'ProgramElement3_Code': root.findtext('.//ProgramElement[3]/Code'),
            'ProgramElement3_Text': root.findtext('.//ProgramElement[3]/Text'),
            'ProgramReference_Code': root.findtext('.//ProgramReference/Code'),
            'ProgramReference_Text': root.findtext('.//ProgramReference/Text'),
            'Appropriation_Code': root.findtext('.//Appropriation/Code'),
            'Appropriation_Name': root.findtext('.//Appropriation/Name'),
            'Appropriation_SymbolID': root.findtext('.//Appropriation/APP_SYMB_ID'),
            'Fund_Code': root.findtext('.//Fund/Code'),
            'Fund_Name': root.findtext('.//Fund/Name'),
            'Fund_SymbolID': root.findtext('.//Fund/FUND_SYMB_ID'),
            'Fund_Obligation': root.findtext('.//FUND_OBLG')
        }
        return data
    except ET.ParseError:
        print(f'Error parsing file: {file_path}')
        return None

def collect_data_from_xml_files(root_folder):
    data_list = []
    error_files = []

    for year_folder in os.listdir(root_folder):
        year_path = os.path.join(root_folder, year_folder)
        if os.path.isdir(year_path):  # Ensure it's a directory
            for filename in os.listdir(year_path):
                if filename.endswith('.xml'):
                    file_path = os.path.join(year_path, filename)
                    data = parse_xml(file_path)
                    if data is not None:
                        data['Year'] = year_folder
                        data_list.append(data)
                    else:
                        error_files.append(file_path)

    return pd.DataFrame(data_list), error_files

# Define the root folder where the year folders are located
root_folder = './temp'

# Collect data and create DataFrame
df, error_files = collect_data_from_xml_files(root_folder)

# Get the current date and time
now = datetime.now()
date_time_str = now.strftime("%Y%m%d_%H%M%S")

# Save the DataFrame to a CSV file with date and time in the filename
output_csv_path = f'output_data_{date_time_str}.csv'
df.to_csv(output_csv_path, index=False)

# Save the error log to a CSV file with date and time in the filename
error_log_path = f'error_log_{date_time_str}.csv'
pd.DataFrame(error_files, columns=['FilePath']).to_csv(error_log_path, index=False)

print(f'Data successfully saved to {output_csv_path}')
print(f'Error log saved to {error_log_path}')
print(f'Successfully parsed XML files: {len(df)}')
print(f'Files with errors: {len(error_files)}')

Data successfully saved to output_data_20250120_125616.csv
Error log saved to error_log_20250120_125616.csv
Successfully parsed XML files: 19329
Files with errors: 0
